### Import Necessary Packages

In [120]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

### Import Basketball Players Data

In [121]:
# Read the data from CSV file
data = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 2 Files\BasketballPlayers.csv")

### Extract Relevant Information

In [122]:
# Extract relevant information
num_players = len(data)
players = range(num_players)
positions = data['Position']
skill_score = skill_scores = data.iloc[:, 2:].sum(axis=1).tolist()

In [123]:
print(num_players)
print(players)
print(positions[:5])
print(skill_score[:5])

150
range(0, 150)
0    G/F
1    G/F
2    G/F
3    G/F
4    F/C
Name: Position, dtype: object
[12, 13, 13, 13, 17]


### Create model

In [124]:
model = gp.Model("BasketballTrainingCamp")

### Decision Varibles

In [125]:
x = model.addVars(players, vtype=GRB.BINARY, name="invite")

### Objective Function

In [126]:
model.setObjective(gp.quicksum(skill_scores[i] * x[i] for i in players), GRB.MAXIMIZE)

### Constraints

In [127]:
# Constraint to limit the total number of invitations to 21
model.addConstr(gp.quicksum(x[i] for i in players) == 21, "TotalInvitations")

<gurobi.Constr *Awaiting Model Update*>

In [128]:
# At least 30% of the invitations should go to guard players
guard_players = [i for i, pos in enumerate(positions) if 'G' in pos]
model.addConstr(gp.quicksum(x[i] for i in guard_players) >= 0.3 * num_players, "GuardPlayers")

<gurobi.Constr *Awaiting Model Update*>

In [129]:
# At least 40% should go to forward/center players
forward_center_players = [i for i, pos in enumerate(positions) if 'F' in pos or 'C' in pos]
model.addConstr(gp.quicksum(x[i] for i in forward_center_players) >= 0.4 * num_players, "ForwardCenterPlayers")

<gurobi.Constr *Awaiting Model Update*>

In [130]:
# Average skill score constraint for each skill
average_skill_score = sum(skill_scores) / num_players
model.addConstr(gp.quicksum(skill_scores[i] * x[i] for i in players) >= 2.05 * num_players, f"AverageSkillScore")

<gurobi.Constr *Awaiting Model Update*>

In [131]:
# If any player from 20-24 (inclusive) is invited, then all players from 72-78 (inclusive) cannot be invited
for i in range(20, 25):
    model.addConstr(gp.quicksum(x[j] for j in range(72, 79)) <= 6 - x[i-1], f"ExcludeIf20to24Invited_{i}")

In [132]:
# If player from 105-114 is invited, at least one player from 45-49 and 65-69 must be invited
model.addConstr(gp.quicksum(x[i] for i in range(105, 115)) <= gp.quicksum(x[i] for i in range(44, 50)) + gp.quicksum(x[i] for i in range(64, 70)), "ConditionalInvitations")

<gurobi.Constr *Awaiting Model Update*>

In [133]:
# At least one player must be invited from each group (1-10, 11-20, ..., 141-150)
group_indices = [(i, min(i + 9, 150)) for i in range(1, 142, 10)]
for group in group_indices:
    model.addConstr(gp.quicksum(x[i] for i in range(group[0]-1, group[1])) >= 1, f"AtLeastOneFromGroup_{group[0]}_{group[1]}")

In [134]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 25 rows, 150 columns and 692 nonzeros
Model fingerprint: 0x8615323c
Variable types: 0 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 0 rows and 22 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 0
No other solutions better than -1e+100

Model is infeasible
Best objective -, best bound -, gap -


In [137]:
# Print the optimal objective function value
if model.status == GRB.OPTIMAL:
    print(f"Optimal objective function value: {model.objVal}")

    # Count the number of guards invited
    num_guards_invited = sum(x[i].x for i in guard_players)
    print(f"Number of guards invited: {num_guards_invited}")

    # Print the list of invited players
    print("Invited players:")
    for i in players:
        if x[i].x > 0.5:
            print(f"Player {i+1} with skill scores {skill_scores[i]}")
else:
    print("Optimal solution not found. Please check your model and constraints.")

Optimal solution not found. Please check your model and constraints.


### (a) What types of decision variables are needed to solve the problem?

Binary Decision Variables are required to represent whether a player is invited to the training camp or not

### (b) How many decision variables must be included in the model?

150 Decision Variables will be used, one to represent each player

### (c) What is the objective function? Defend why this is an appropriate choice.

The Objective Function will attempt to maximize the skill level of the invited players. 

This is an appropriate choice as the goal of the model is to select 21 players that have the highest skill rating among the 150 options.

### (d) Write down the following constraint in mathematical notation but only for player 72: If any player from 20-24 (inclusive) is invited, all players from 72-78 (inclusive) cannot be.

X20 + X21 + X22 + X23 + X24 <= 4 - 4X72

### (e) Write down the following constraint in mathematical notation: At least 30% of the invitations should go to players that can play the guard position (G, G/F).

∑i∈G xi ≥ 0.3×150

### (f) What is the optimal objective function value?

### (g) How many guards (G, G/F) are invited to the training camp?

### (h) What is the smallest number of training camp invitations that can be sent before the model yields an infeasible solution? What constraint(s) cannot be satisfied?

### (i) Describe (do not implement) the challenge of modifying your solution approach to ensure that players with a total score of 12 or under would not be invited to training camp.

### (j) What do you perceive as a problem with Victor’s approach of choosing participants?